In [1]:
import os
import time
import sys
import clr
import numpy as np

# Add References to .NET libraries
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.Benchtop.DCServoCLI.dll.")


from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.Benchtop.DCServoCLI import *
from System import Decimal  # Required for real units

In [2]:
DeviceManagerCLI.BuildDeviceList()
# create new device
serial_no = "101334424"  # Replace this line with your device's serial number

device = BenchtopDCServo.CreateBenchtopDCServo(serial_no)

# Connect, begin polling, and enable
device.Connect(serial_no)
time.sleep(0.25)  # wait statements are important to allow settings to be sent to the device

# Get Device Information and display description
device_info = device.GetDeviceInfo()
print(device_info.Description)

# Get the channel for the device
x_channel = device.GetChannel(1)  # Returns a benchtop channel object
y_channel = device.GetChannel(2)

# Start Polling and enable channel
x_channel.StartPolling(250)
y_channel.StartPolling(250)
time.sleep(0.25)
x_channel.EnableDevice()
y_channel.EnableDevice()
time.sleep(0.25)

# Check that the settings are initialised, else error.
if not x_channel.IsSettingsInitialized() or not y_channel.IsSettingsInitialized():
    x_channel.WaitForSettingsInitialized(10000)  # 10 second timeout
    y_channel.WaitForSettingsInitialized(10000)
    assert device.IsSettingsInitialized() is True

# Load the motor configuration on the channel
x_config = x_channel.LoadMotorConfiguration(x_channel.DeviceID)
y_config = y_channel.LoadMotorConfiguration(y_channel.DeviceID)

# Read in the device settings
dev_settings = x_channel.MotorDeviceSettings

# Get the Homing Params
x_home_params = x_channel.GetHomingParams()
y_home_params = y_channel.GetHomingParams()

x_home_params.Velocity = Decimal(2.0)
y_home_params.Velocity = Decimal(2.0)

x_channel.SetHomingParams(x_home_params)
y_channel.SetHomingParams(y_home_params)

M30X/M30XY


In [23]:
print("Homing X Channel")
x_channel.Home(60000)  # 60 second timeout

Homing X Channel


In [25]:
x_vel_params = x_channel.GetVelocityParams()
y_vel_params = y_channel.GetVelocityParams()

x_vel_params.Acceleration = Decimal(5.0)
x_vel_params.MaxVelocity = Decimal(2.4)
y_vel_params.Acceleration = Decimal(5.0)
y_vel_params.MaxVelocity = Decimal(2.4)

x_channel.SetVelocityParams(x_vel_params)
y_channel.SetVelocityParams(y_vel_params)

In [57]:
x_channel.MoveTo(Decimal(float(15)), 20000)

In [58]:
dec = x_channel.DevicePosition

In [59]:
dec

In [60]:
print(dec)

15.0002


In [61]:
y_channel.StopPolling()
x_channel.StopPolling()
device.Disconnect()